# Milestone One

### Imports
This process relys heavily on requests and BeautifulSoup to crawl and scrape html

In [761]:
import requests
import bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas import DataFrame as df 
from requests.exceptions import MissingSchema

### Methods

the GetValidUrl() Method pulls a websites url and varifies the sites validity
When the application gets to a more self sustainable state this method can be implemented to validate given URL's for fast development

In [762]:
def getValidUrl(): #in this method a user inputs a URL for validation
    isVal = False
    while isVal == False:   
        try: 
            unVal = requests.get(input("Please enter an URL (includeHTTP://)':")) #url requires http://
            print("URL is Valid")
            isVal = True #ensures a url is valid for crawling/scraping
            return unVal
        except requests.ConnectionError as exception:
            print("URL does not exist on Internet") #if a correctly entered URL has no site
        except MissingSchema as exception:
            print("URL is not complete") #This idicates a URL was missing schema needed for the method (http://)



per_base() is a method that returns a dictionary with keys, but no values. This will be the basis for the persistant database we will keep event information in. Each key represents a collumn in a database. When parsing a webpage data will be stored in this same dictionary format. 

In [773]:
def per_base():
    persistant_dic_base = {
        'Event Title' :"",
        'Topline': "",
        'Headliner':"",
        'Openers':"",
        'Date(s)':"",
        'Time(s)':"",
        'Price/Admission':'',
        'Age Restriction':'',
        'Event Description':'',
        'Venue':'',
        'Venue Info':'',
        'Street Name':'',
        'Address Line 2':',',
        'City':'',
        'State':'',
        'Postal Code':'',
        'Event Image URL':"",
        'Visit Event Website':'',
        'Venue Info':'',
        'Location Link':'',
        'Get Tickets':'',
        'URL':''}
    return persistant_dic_base #this will be used as a template for dictionaries that will be used in the scrapign process

### The next two methods assist with web crawling

webBuilder() is a method that takes a list of unparsed html code, and is specificly used for clensing data for Dada Dallas. 

Each item in the list of unparsed code contains a link to a specific events page within it. 

This method initiates a parsing process to find the a link in the html code, and appends it to a new list of links. 

It returns a list of links to event pages, for further crawling and scraping.

In [774]:
def webBuilder(page_list): #this method finds url's for events listed on the main callender page
    events_list = []
    for i in page_list:
        cont = i.contents[0]
        link = cont['href']
        events_list.append(link)
    return events_list

visit_dallas_pages() is a method that takes an BeautifulSoup Element and a website URL. This method is specific to the Visit Dallas website.

This method acomplishes the same task for webBuilder() but creates the list of unparsed html code containing links in the method. 

It takes a base URL beacuse the links that are parsed from the website contain the later half of the website. 

This method adds the bass URL to the parsed links to create a list of functional URL's that can be used for further parsing and scraping.

In [775]:
def visit_dallas_pages(souped, base):
    pages=[]
    base2 =""
    soupy = souped.find_all('a', class_="card event-card event-card--premiere")
    for i in soupy:
        base2 = base + i.attrs['href']
        pages.append(base2)
    soupy = souped.find_all('a', class_= "card event-card")
    for i in soupy:
        base2 = base + i.attrs['href']
        pages.append(base2)
    return pages

#### Developer Notes: 
These two methods were constructed to help with the crawling/scraping of specific websites used in testing. In further iteration we plan on creating a single method that is able to crawl and return sites for scraping.



### Scraping Methods

dada_scrape() is a method that takes in a list of events and a dictionary. It ultimately returns a dataframe of with data from each page scraped. This method is specific to collecting data from the Dada Dallas website

The dictionary is used as a template for how information from each page will be stored. 

For each page in the given list, it searcehs each page for various data, scrapes it, and adds it to a dictionary. 

At the end of scraping for a page, the filled out dictionary is added to a list. 

At the end of the method, the list of dictionaries is converted into a dataframe that is returned.

In [776]:
def dada_scrape(cal_events, pbas=dict):
    page_deets = []
    for link in cal_events:
        cont_list = pbas.copy()
        on_sale = True
        s_page = requests.get(link)
        sub_soup = BeautifulSoup(s_page.text, 'html.parser')
        sub_parse = sub_soup.find('meta', property ="og:title")
        cont_list.update([('Event Title', sub_parse.attrs['content'])])
        sub_parse = sub_soup.find('section', class_="topline-info presented-by")
        cont_list.update([('Topline', sub_parse.get_text())])
        sub_parse = sub_soup.find('h1', class_="event-name headliners")
        cont_list.update([('Headliner', sub_parse.get_text())])
        sub_parse = sub_soup.find('h2', class_="supports")
        if type(sub_parse) is type(None):
            cont_list.update([('Openers', 'NA')])
        else:
            cont_list.update([('Openers', sub_parse.get_text())])
        sub_parse = sub_soup.find('span', class_="dates")
        cont_list.update([('Date(s)', sub_parse.get_text())])
        sub_parse = sub_soup.find('span', class_="start")
        cont_list.update([('Time(s)', sub_parse.get_text())])
        sub_parse = sub_soup.find('span', class_="sales-ended inactive")
        if type(sub_parse) is type(None):
            sub_parse = sub_soup.find('span', class_="price-range")
            cont_list.update([('Price/Admission', sub_parse.get_text())])
        else:
            cont_list.update([('Price/Admission', sub_parse.get_text())])
            on_sale = False
        sub_parse = sub_soup.find('section', class_="age-restriction all-ages")
        cont_list.update([('Age Restriction', sub_parse.get_text())])
        sub_parse = sub_soup.find('article', class_="event-description")
        cont_list.update([('Event Description', sub_parse.contents[1].text + " " +sub_parse.contents[3].text)])
        sub_parse = sub_soup.find('address', class_="venue-info")
        cont_list.update([('Venue', sub_parse.contents[2])])
        sub_parse = sub_soup.find('meta', property="og:street-address")
        cont_list.update([('Street Name', sub_parse.attrs['content'])])
        sub_parse = sub_soup.find('meta', property="og:locality")
        cont_list.update([('City', sub_parse.attrs['content'])])
        sub_parse = sub_soup.find('meta', property="og:region")
        cont_list.update([('State', sub_parse.attrs['content'])])
        sub_parse = sub_soup.find('meta', property="og:postal-code")
        cont_list.update([('Postal Code', sub_parse.attrs['content'])])
        sub_parse = sub_soup.find('img')
        cont_list.update([('Event Image URL', sub_parse.attrs['src'])])
        cont_list.update([('URL',link)])
        
        if on_sale == True:
            sub_parse = sub_soup.find('section', class_ = 'ticket-price')
            for i in sub_parse.contents:
                if type(i) == bs4.element.Tag and 'href' in i.attrs:
                    cont_list.update([('Get Tickets', i.attrs['href'])])
                elif type(i) == bs4.element.Tag:
                        for j in i.contents:
                            if type(j)== bs4.element.Tag and 'href' in j.attrs: 
                                cont_list.update([('Get Tickets', j.attrs['href'])])
        else:
            cont_list.update([('Get Tickets', cont_list['Price/Admission'])])
            
            
        page_deets.append(cont_list)
        
    spp = df.from_records(page_deets)
    return spp

vd_scraper() is a method that takes in a list of events, a bass url, and a dictionary. This scrapign method is speciclly for the Visit Dallas Website

The dictionary is used as a template for how information from each page will be stored. 

For each page in the given list, it searcehs each page for various data, scrapes it, and adds it to a dictionary. 

At the end of scraping for a page, the filled out dictionary is added to a list. 

At the end of the method, the list of dictionaries is converted into a dataframe that is returned.

In [777]:
def vd_scraper(vd_events, base, pbas=dict):
    page_deets = []
    for link in vd_events:
        pg_con = pbas.copy()
        s_page = requests.get(link)
        sub_soup = BeautifulSoup(s_page.text, 'html.parser')
        sub_parse = sub_soup.find('h1', class_="event-perma-card__title")
        pg_con.update([('Event Title', sub_parse.get_text())])
        sub_parse = sub_soup.find('div', class_="event-perma-card__date-tag")
        span =""
        for i in sub_parse.contents:   
            if i != '\n':
                if span =="":
                    span = i.get_text()
                else:
                    span = span + " " + i.get_text()
        pg_con.update([('Date(s)', span)])
        sub_parse = sub_soup.find_all('p', class_="event-perma-content__text")
        #print(sub_parse[1].contents[1].get_text())
        pg_con.update([('Venue', sub_parse[1].contents[1].get_text())])
        pg_con.update([('Street Name', sub_parse[1].contents[4])])
        if len(sub_parse[1].contents) > 10:
            pg_con.update([("Address Line 2", sub_parse[1].contents[6])])
            ay = sub_parse[1].contents[8]
            pg_con.update([('City', ay.split()[0])])
            pg_con.update([('State',ay.split()[1])])
            pg_con.update([('Postal Code',ay.split()[2])])
        else:
            pg_con.update([("Address Line 2", "NA")])
            ay = sub_parse[1].contents[6]
            pg_con.update([('City', ay.split()[0])])
            pg_con.update([('State',ay.split()[1])])
            pg_con.update([('Postal Code',ay.split()[2])])
        oi = sub_parse[2].contents[0]
        oii = oi.split("  ")
        pg_con.update([('Time(s)', oii[len(oii)-1])])
        if len(sub_parse) > 3:
            pg_con.update([('Price/Admission', sub_parse[3].contents[0])])
        else:
            pg_con.update([('Price/Admission', "")])
        sub_parse = sub_soup.find('div', class_='wysiwyg')
        pg_con.update([('Event Description', sub_parse.get_text()[1:])])
        sub_parse = sub_soup.find('div', class_='event-perma-card__meta-ctas-wrap')
        pg_con.update([('Get Tickets',"")])
        pg_con.update([('Visit Event Website',"")])
        pg_con.update([('Venue Info',"")])
        for i in sub_parse.contents:
            if type(i) == bs4.element.Tag:
                for j in i.contents:
                    if type(j) == bs4.element.Tag:
                        pg_con.update([(j.get_text(),(base + j['href']))])
        sub_parse = sub_soup.find('p', class_='event-perma-card__meta-item pre-icon--pin') 
        if type(sub_parse) == bs4.element.Tag:
            pg_con.update([('Location Link', sub_parse.contents[0]['href'])])
        sub_parse = sub_soup.find('div', class_='event-perma-card__image')
        pg_con.update([('Event Image URL', sub_parse.contents[1]['src'])])
        
        pg_con.update([('URL',link)])        
        page_deets.append(pg_con)
    spp = df.from_records(page_deets)    
    return spp    

#### Developer Notes: 
These two methods were constructed to help with the scraping of specific websites used in testing. In further development this process is aimed to be streamlined to more flexibly integrate with sites out there.


### Crawling and Scraping

In this milestone we look at two websites that contain information about events in Dallas: Dadadallas.com and visitdallas.com. 

Club Dada , also known as Dada Dallas, is a popular venue and in Deep Ellum. VisitDallas is an independent, not-for-profit organization that promotes Dallas as a business and pleasure destination. 

Both of these sites have pages of events, and in this milestone we developed a web scraper that can collect information from these websites.



In this first cell we scrape data for Dada Dallas events. It takes the webpage and parses it using beautiful soup and the methods created earlier. It finds all the active calendar events, collects their URL's, and then scrapes each event page for information. Each pages information is stored in a dictionary, added to a list of dictionaries, which is converted into a dataframe.

In [778]:
site = requests.get("https://www.dadadallas.com/calendar/")
site.encoding = 'ISO-885901'
soup = BeautifulSoup(site.text, 'html.parser')
page_list = soup.find_all(class_='event-name headliners')
sub_pages = webBuilder(page_list)
pbas = per_base()
j1 = dada_scrape(sub_pages, pbas)


In the next call we scrape data from the VisitDallas website. This functions similary to the previous cell, but uses its own case specific methods for scraping.

At the end of the cell the scraping method returns a dataframe with information from all events scraped.

In [779]:
site2 = requests.get("https://www.visitdallas.com/things-to-do/events/arts-culture-in-dallas/index.html")
site2.encoding = 'ISO-885901'
soup2 = BeautifulSoup(site2.text, 'html.parser')
su_parse = soup2.find_all('a', class_= "card event-card")
base = "https://www.visitdallas.com"
dal_list = visit_dallas_pages(soup2, base)
pbas = per_base()
j2 = vd_scraper(dal_list, base, pbas)

Now we have two dataframes with event data from different websites. 

Because we used the same dictionary format to save data for these events, the two dataframes can easily be appended into a larger persistant database.

Once we have this centralized database, it can be exported into a csv for further clensing/use/

In [780]:
j3 = j1.append(j2)
df.to_csv(j3, "PDBASE.csv", sep = ",", index = False)